In [1]:
from typing import Callable,Dict, Literal, Optional
from langchain_ollama import ChatOllama
import re 
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
import uuid

In [2]:
MODEL_NAME = 'qwen3:latest'

In [3]:
def booking_handler(request: str) -> str:
    """
    Handles booking requests for flights and hotels.
    """
    print("-------------------------- Booking Handler Called ----------------------------")
    return f"Booking action for '{request}' has been simulated."

def info_handler(request: str) -> str:
    """
    Handles general information requests.
    """
    print("-------------------------- Info Handler Called ----------------------------")
    return f"Information request for '{request}'. Result: Simulated information retrieval."

def unclear_handler(request: str) -> str:
    """Handles requests that couldn't be delegated."""
    return f"Coordinator could not delegate request: '{request}'. Please clarify."

In [4]:
booking_handler("some request")

-------------------------- Booking Handler Called ----------------------------


"Booking action for 'some request' has been simulated."

In [5]:
# -- Minimal Agent Abstraction -------------
class Agent:
    def __init__(self, name:str, description:str,tool: Callable[[str],str]):
        self.name = name
        self.description = description
        self.tool = tool
    
    def run(self,request:str) -> str:
        return self.tool(request)
    
booking_agent = Agent(
    name = "Booker",
    description= "Handles all the flight and hotel booking request",
    tool = booking_handler,
)

info_agent = Agent(
    name= "Info",
    description= "Provdies general information and answers user questions",
    tool= info_handler,
)

In [6]:
booking_agent.run("my requst")

-------------------------- Booking Handler Called ----------------------------


"Booking action for 'my requst' has been simulated."

In [7]:
booking_agent.name

'Booker'

In [8]:
AGENT_REGISTRY: Dict[str, Agent] = {
    "Booker": booking_agent,
    "Info": info_agent,
}

In [9]:
# LLM Setup

llm_text = ChatOllama(
    model= MODEL_NAME,
    temperature=0,
)

llm_json = ChatOllama(
    model=MODEL_NAME,
    temperature=0,
    format = "json",
)

def strip_think(text:str) -> str:
    """
    Remove any < think> blocks
    """
    
    return re.sub(r"<think>.*?</think>\s*", "", text, flags=re.DOTALL | re.IGNORECASE)
    

In [10]:
# to invoke chat using llm
messages = [
    ("human", "Tell me about yourself")
]

In [11]:
chat_text = llm_text.invoke(messages)

In [12]:
chat_text.content

"<think>\nOkay, the user asked me to tell them about myself. I need to make sure I cover the key points without being too technical. Let me start by introducing my name, Qwen, and my role as a large language model. I should mention my development by Alibaba Cloud and my purpose to assist with various tasks.\n\nI should highlight my capabilities, like answering questions, creating content, and solving problems. It's important to note that I can handle multiple languages and adapt to different scenarios. Also, I need to mention my training data up to 2024, which gives me up-to-date knowledge.\n\nBut wait, I should avoid any technical jargon. Maybe explain that I can understand and generate text in various formats, like articles or code. Also, I should emphasize my ability to learn from interactions and improve over time. Oh, and I should invite the user to ask questions or request help with specific tasks. Let me structure this in a friendly and conversational way without using markdown.

In [13]:
strip_think(chat_text.content)

"Hello! I'm Qwen, a large language model developed by Alibaba Cloud. My main purpose is to assist with a wide range of tasks, such as answering questions, creating content, solving problems, and more. I can understand and generate text in multiple languages, adapting to different scenarios and user needs. My training data is up to 2024, which means I can provide information and insights based on the latest knowledge available. I'm designed to be helpful, efficient, and friendly, and I'm always here to assist you with whatever you need! How can I help you today? 😊"

In [14]:
chat_json = llm_json.invoke(messages)
chat_json

AIMessage(content='{"error": "Forbidden", "message": "You are not allowed to access this resource."}', additional_kwargs={}, response_metadata={'model': 'qwen3:latest', 'created_at': '2025-10-10T15:51:11.897742899Z', 'done': True, 'done_reason': 'stop', 'total_duration': 609065589, 'load_duration': 47388843, 'prompt_eval_count': 12, 'prompt_eval_duration': 3106525, 'eval_count': 21, 'eval_duration': 558065032, 'model_name': 'qwen3:latest'}, id='run--52d5fe27-58c0-4729-b89a-57b43024ae6f-0', usage_metadata={'input_tokens': 12, 'output_tokens': 21, 'total_tokens': 33})

In [15]:
print(chat_json.content)

{"error": "Forbidden", "message": "You are not allowed to access this resource."}


In [16]:
# Implementing Routing

class RouteDecision(BaseModel):
    route: Literal["Booker", "Info", "Unclear"] = Field(
        description="Which sub-agent should handle this request: 'Booker' for booking flights/hotels; 'Info' for general questions; 'Unclear' if ambiguous."
    )
    reason: str = Field(description="Brief reason for the decision.")

In [17]:
test_route_decsion = RouteDecision(route= "Booker",reason="not")

In [18]:
test_route_decsion.route

'Booker'

In [19]:
ROUTER_SYSTEM = """You are the Coordinator. Your ONLY task:
- Analyze the user's request and choose the proper sub-agent.
- Never answer the user directly.
- Return STRICT JSON with keys: route, reason.

Routing rules:
- If the request is about booking flights or hotels -> route = "Booker".
- Otherwise, if it's a general information question -> route = "Info".
- If it's ambiguous or cannot be routed -> route = "Unclear".
"""

ROUTER_HUMAN = """User request:
{request}

Return JSON only, like:
{{"route": "Booker", "reason": "It's about hotel booking"}}
"""

router_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", ROUTER_SYSTEM),
        ("human", ROUTER_HUMAN),
    ]
)


In [20]:
router_chain = router_prompt | llm_json|JsonOutputParser(pydantic_object=RouteDecision)


In [21]:
class Coordinator:
    def __init__(self):
        self.session_id = str(uuid.uuid4())

    def route(self, request: str) -> RouteDecision:
        decision: RouteDecision = router_chain.invoke({"request": request})
        return decision

    def handle(self, request: str) -> str:
        print(f"\n--- Coordinator handling request: '{request}' ---")
        try:
            decision = self.route(request)
            print(f"Routing -> {decision["route"]} | Reason: {decision["reason"]}")

            if decision["route"] in AGENT_REGISTRY:
                agent = AGENT_REGISTRY[decision["route"]]
                return agent.run(request)
            else:
                return unclear_handler(request)
        except Exception as e:
            return f"An error occurred while processing your request: {e}"
    
    

In [22]:
coordinator = Coordinator()
c = coordinator.handle("Book me a hotel in Paris.")


--- Coordinator handling request: 'Book me a hotel in Paris.' ---
Routing -> Booker | Reason: It's about hotel booking
-------------------------- Booking Handler Called ----------------------------


In [24]:
def main():
    print("--- Ollama deepseek-r1 Routing Example (Auto-Flow Style) ---")
    print("Note: Requires Ollama running locally with `deepseek-r1:7b` pulled.\n")

    coordinator = Coordinator()

    # Example Usage
    result_a = coordinator.handle("Book me a hotel in Paris.")
    print(f"Final Output A: {result_a}")

    result_b = coordinator.handle("What is the highest mountain in the world?")
    print(f"Final Output B: {result_b}")

    result_c = coordinator.handle("Tell me a random fact.")  # Should go to Info
    print(f"Final Output C: {result_c}")

    result_d = coordinator.handle("Find flights to Tokyo next month.")  # Should go to Booker
    print(f"Final Output D: {result_d}")


if __name__ == "__main__":
    main()

--- Ollama deepseek-r1 Routing Example (Auto-Flow Style) ---
Note: Requires Ollama running locally with `deepseek-r1:7b` pulled.


--- Coordinator handling request: 'Book me a hotel in Paris.' ---
Routing -> Booker | Reason: It's about hotel booking
-------------------------- Booking Handler Called ----------------------------
Final Output A: Booking action for 'Book me a hotel in Paris.' has been simulated.

--- Coordinator handling request: 'What is the highest mountain in the world?' ---
Routing -> Info | Reason: It's a general information question about geography
-------------------------- Info Handler Called ----------------------------
Final Output B: Information request for 'What is the highest mountain in the world?'. Result: Simulated information retrieval.

--- Coordinator handling request: 'Tell me a random fact.' ---
Routing -> Info | Reason: The request is for a random fact, which falls under general information.
-------------------------- Info Handler Called -------------